# Retrieve DNS queries from zeek logs

In [1]:
import pandas as pd
import numpy as np 
import binascii # Binary to Ascii 
from pathlib import Path
from datetime import datetime
import pickle
import io
import glob
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
fields = ['ts',	'uid',	'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',	'proto', 'trans_id', 'rtt',	'query', 'qclass', 'qclass_name', 'qtype', 'qtype_name', 'rcode']

In [3]:
def zeekToCsv(conlog: str, fieldz: list):
    outcsv = 'dnstemp.csv'
    with open(conlog) as file:
        lines = file.read().splitlines()
        with open(outcsv,"w") as outfile:
            for line in lines:
                if line[0] == '#':
                    continue
                l = (line.split())[0:10]
                tmp = ",".join(l)
                outfile.write(tmp + '\n')

In [4]:
logs = glob.glob('zeeklogs/*/dns.log')
dnslogs = pd.DataFrame()
for log in logs:
    print(log)
    zeekToCsv(log, fields)
    dnslogs = pd.concat([dnslogs, pd.read_csv("dnstemp.csv", names=fields)])  

zeeklogs/jul27/dns.log
zeeklogs/jul26/dns.log
zeeklogs/jul18/dns.log
zeeklogs/aug9/dns.log
zeeklogs/aug1/dns.log
zeeklogs/aug6/dns.log
zeeklogs/aug3/dns.log
zeeklogs/jul25/dns.log
zeeklogs/jul14/dns.log
zeeklogs/jul28/dns.log


In [5]:
dnslogs

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,trans_id,rtt,query,qclass,qclass_name,qtype,qtype_name,rcode
0,1.658937e+09,Co4nWB171V1HTI91th,192.168.70.4,63449,8.8.8.8,53,udp,26037,0.043278,slack.com,NaN,NaN,NaN,NaN,NaN
1,1.658937e+09,CFdk5F2EPqNymSIRii,192.168.70.160,57127,192.168.70.4,53,udp,37520,0.045227,slack.com,NaN,NaN,NaN,NaN,NaN
2,1.658937e+09,CjnWloPD3el7oZkMl,192.168.70.160,57127,8.8.8.8,53,udp,37520,0.034362,slack.com,NaN,NaN,NaN,NaN,NaN
3,1.658937e+09,CLBuTK2EXeHe6XPAsl,192.168.70.4,62033,8.8.8.8,53,udp,46555,0.498743,a6f903166400000000788f76e1ef4cf991864a6373f444...,NaN,NaN,NaN,NaN,NaN
4,1.658937e+09,CoPffy1t0UhFQtLIo1,192.168.70.101,42352,192.168.70.4,53,udp,63304,0.500607,a6f903166400000000788f76e1ef4cf991864a6373f444...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18100,1.659050e+09,CK8ZjA1UgsHtMvR7X1,fe80::a4f2:cba0:d14:93ca,5353,ff02::fb,5353,udp,0,-,_googlecast._tcp.local,NaN,NaN,NaN,NaN,NaN
18101,1.659050e+09,CK8ZjA1UgsHtMvR7X1,fe80::a4f2:cba0:d14:93ca,5353,ff02::fb,5353,udp,0,-,_googlecast._tcp.local,NaN,NaN,NaN,NaN,NaN
18102,1.659050e+09,C2R2Cftz8O2OiZb1j,192.168.70.160,5353,224.0.0.251,5353,udp,0,-,_googlecast._tcp.local,NaN,NaN,NaN,NaN,NaN
18103,1.659050e+09,C2R2Cftz8O2OiZb1j,192.168.70.160,5353,224.0.0.251,5353,udp,0,-,_googlecast._tcp.local,NaN,NaN,NaN,NaN,NaN


In [6]:
contain_values = dnslogs[dnslogs['query'].str.contains('nowstudios.xyz')]
x = contain_values['uid'].tolist()

In [7]:
pickle.dump(x, open('dnstunneluids', 'wb'))